Your Name:  Gregory Roberts

Submission Instruction:
* Instruction: make a copy of this CoLab file and share it with me (Yifeng.Zhu@maine.edu).
* Deadline: Midnight, Sunday, April 2


In [3]:
!pip install mpi4py

     |████████████████████████████████| 2.5 MB 5.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.3-cp37-cp37m-linux_x86_64.whl size=2185277 sha256=f3569827b44add27c4d892f94f7224ec2d37f81d0dd2f48950c0c5ae045887e2
  Stored in directory: /root/.cache/pip/wheels/7a/07/14/6a0c63fa2c6e473c6edc40985b7d89f05c61ff25ee7f0ad9ac
Successfully built mpi4py


#Question 1: Point-to-point communication

Is there any problem in the following code? If so, how to fix it?

In [ ]:
%%file q1.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank==1:
    data_send= "a"
    destination_process = 2
    source_process = 2

    comm.send(data_send, dest=destination_process)
    data_received = comm.recv(source=source_process)
    
    print ("sending data %s " %data_send + \
           "to process %d" %destination_process)
    print ("data received = %s" %data_received)

if rank==2:
    data_send= "b"
    destination_process = 1
    source_process = 1

    comm.send(data_send, dest=destination_process)
    data_received = comm.recv(source=source_process)
    
    print ("sending data %s :" %data_send + \
           "to process %d" %destination_process)
    print ("data received = %s" %data_received)


Writing q1.py


In [ ]:
!mpirun --allow-run-as-root -n 4 python q1.py

sending data a to process 2
data received = b
sending data b :to process 1
data received = a


**Your answer:**  Changed the code so that the comm.send is before the comm.recv for both rank processes.


# Question 2: Fix the bug of the following code

The following code is to implement a parallel matrix vector product. However, there are a large different between the parallel implementation and the result produced by numpy.dot. What is wrong with the code and how to fix it?

In [ ]:
# This code is used to generate the vector and matrix that will be used in the
# next section. These are then stored in files so that the values do not change
# each time the process iterates through the rank.

import numpy as np
import os

n = 400

x = np.random.rand(n)     # Generate a vector     
A = np.random.rand(n, n)  # Generate a nxn matrix

np.savetxt(fname="x_array.csv", delimiter=",", X=x)
np.savetxt(fname="A_matrix.csv", delimiter=",", X=A)



Numpy Version is  1.21.5


In [1]:
%%file matvec.py
import numpy as np
from mpi4py import MPI
import os

# Parallel matrix-vector product
def matvec(comm, A, x):
    size = comm.Get_size()
    rank = comm.Get_rank()
    m = A.shape[0] // size # local rows
    # every process gets a part of the data
    y_part = np.dot(A[rank * m:(rank+1)*m, :], x) 
    # container for the result    
    y = np.zeros_like(x, dtype='double')     
    # collect results from the pool, write them to container y      
    comm.Allgather([y_part,  MPI.DOUBLE], [y, MPI.DOUBLE])    
    return y

n = 400
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

print('comm.Get_size() ', comm.Get_size())
#x = np.random.rand(n)     # Generate a vector     
#A = np.random.rand(n, n)  # Generate a nxn matrix
x = np.loadtxt(fname="x_array.csv", delimiter=",")
A = np.loadtxt(fname="A_matrix.csv", delimiter=",")

y_mpi = matvec(comm, A, x) # y_mpi = A * x

if rank == 0: # check 
  y = np.dot(A, x)      
  # compare the local and MPI results   
  # The output should be a very small value 
  print("sum(y - y_mpi) = ", (y - y_mpi).sum()) 



Writing matvec.py


In [4]:
!mpirun --allow-run-as-root -n 4 python3 matvec.py

comm.Get_size()  4
comm.Get_size()  4
comm.Get_size()  4
comm.Get_size()  4
Traceback (most recent call last):
  File "matvec.py", line 25, in <module>
    x = np.loadtxt(fname="x_array.csv", delimiter=",")
  File "/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py", line 1067, in loadtxt
    fh = np.lib._datasource.open(fname, 'rt', encoding=encoding)
  File "/usr/local/lib/python3.7/dist-packages/numpy/lib/_datasource.py", line 193, in open
    return ds.open(path, mode, encoding=encoding, newline=newline)
  File "/usr/local/lib/python3.7/dist-packages/numpy/lib/_datasource.py", line 533, in open
    raise IOError("%s not found." % path)
OSError: x_array.csv not found.
Traceback (most recent call last):
  File "matvec.py", line 25, in <module>
    x = np.loadtxt(fname="x_array.csv", delimiter=",")
  File "/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py", line 1067, in loadtxt
    fh = np.lib._datasource.open(fname, 'rt', encoding=encoding)
  File "/usr/local/lib/pyth

**Your answer:**  I tried a number of different processes. I tried to use numpy.matmul, @, *, etc. I tried to split the numpy.dot into separate processes, similar to what was being done in matvec.

I found that for each rank that the A and x arrays were loaded with new numbers for the matvec function. The y dot production would only run once when the rank is 0 (zero).

By using static values stored in files, and then read in. The values stay the same each time, and y_mpi is not loaded with new random values each time the process iterates through the rank. Which produces a smaller value between y and y_mpi.

